# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

05-20-2022


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-05-21 04:20:58,33.93911,67.709953,179624,7695,NaN,NaN,Afghanistan,461.422181,4.283949
1,NaN,NaN,NaN,Albania,2022-05-21 04:20:58,41.15330,20.168300,275732,3497,NaN,NaN,Albania,9581.346862,1.268260
2,NaN,NaN,NaN,Algeria,2022-05-21 04:20:58,28.03390,1.659600,265847,6875,NaN,NaN,Algeria,606.250118,2.586074
3,NaN,NaN,NaN,Andorra,2022-05-21 04:20:58,42.50630,1.521800,42572,153,NaN,NaN,Andorra,55098.686339,0.359391
4,NaN,NaN,NaN,Angola,2022-05-21 04:20:58,-11.20270,17.873900,99287,1900,NaN,NaN,Angola,302.093928,1.913644


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/11/22,5/12/22,5/13/22,5/14/22,5/15/22,5/16/22,5/17/22,5/18/22,5/19/22,5/20/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,179131,179169,179203,179242,179267,179321,179328,179477,179597,179624
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,275440,275485,275534,275574,275615,275621,275688,275732,275732,275732
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,265806,265808,265814,265816,265818,265823,265828,265834,265841,265847


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/11/22,5/12/22,5/13/22,5/14/22,5/15/22,5/16/22,5/17/22,5/18/22,5/19/22,5/20/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7686,7686,7686,7687,7690,7691,7691,7691,7695,7695
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3497,3497,3497,3497,3497,3497,3497,3497,3497,3497
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6875,6875,6875,6875,6875,6875,6875,6875,6875,6875


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/11/22,5/12/22,5/13/22,5/14/22,5/15/22,5/16/22,5/17/22,5/18/22,5/19/22,5/20/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/11/22,5/12/22,5/13/22,5/14/22,5/15/22,5/16/22,5/17/22,5/18/22,5/19/22,5/20/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,15853,15860,15863,15863,15863,15870,15871,15873,15885,15891
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,55807,55831,55862,55862,55862,55916,55941,55960,55996,56031
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5677,5678,5681,5681,5681,5688,5688,5691,5691,5692


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/11/22,5/12/22,5/13/22,5/14/22,5/15/22,5/16/22,5/17/22,5/18/22,5/19/22,5/20/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,216,216,216,216,216,216,216,216,216,216
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,680,680,681,681,681,681,681,681,681,681
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,98,98,98,98,98,98,98,98,98,98


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
675,1001.0,Autauga,Alabama,US,2022-05-21 04:20:58,32.539527,-86.644082,15891,216,NaN,NaN,"Autauga, Alabama, US",28443.322773,1.359260
712,1075.0,Lamar,Alabama,US,2022-05-21 04:20:58,33.779950,-88.096680,3748,65,NaN,NaN,"Lamar, Alabama, US",27149.583484,1.734258
713,1077.0,Lauderdale,Alabama,US,2022-05-21 04:20:58,34.901719,-87.656247,24236,397,NaN,NaN,"Lauderdale, Alabama, US",26136.375891,1.638059


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,83237592,1002020,0.0
India,43134145,524348,0.0
Brazil,30762413,665722,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,83237592,1002020,0.0,82235572.0,2022-05-21 04:20:58,37.934269,-91.444083
India,43134145,524348,0.0,42609797.0,2022-05-21 04:20:58,23.088275,81.806127
Brazil,30762413,665722,0.0,30096691.0,2022-05-21 04:20:58,-12.669522,-48.480493
France,29541498,148818,0.0,29392680.0,2022-05-21 04:20:58,6.430808,-34.730285
Germany,26040460,138324,0.0,25902136.0,2022-05-21 04:20:58,51.571844,10.277881


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,9449915,91044,0.0
Texas,6893736,88303,0.0
Florida,6101783,74331,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,9449915,91044,0.0,9358871.0,2022-05-21 04:20:58,37.843962,-120.728594
Texas,6893736,88303,0.0,6805433.0,2022-05-21 04:20:58,31.660643,-98.653069
Florida,6101783,74331,0.0,6027452.0,2022-05-21 04:20:58,28.940755,-82.700744
New York,5355624,68777,0.0,5286847.0,2022-05-21 04:20:58,42.544151,-75.474183
Illinois,3249534,36194,0.0,3213340.0,2022-05-21 04:20:58,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,2929950,32074,0.0
Arizona,Maricopa,1288395,17342,0.0
Florida,Miami-Dade,1242434,10957,0.0
Illinois,Cook,1211998,14401,0.0
Texas,Harris,1037219,10981,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,2929950,32074,0.0,2897876.0,2022-05-21 04:20:58,34.308284,-118.228241,6037.0
Arizona,Maricopa,1288395,17342,0.0,1271053.0,2022-05-21 04:20:58,33.348359,-112.491815,4013.0
Florida,Miami-Dade,1242434,10957,0.0,1231477.0,2022-05-21 04:20:58,25.611236,-80.551706,12086.0
Illinois,Cook,1211998,14401,0.0,1197597.0,2022-05-21 04:20:58,41.841448,-87.816588,17031.0
Texas,Harris,1037219,10981,0.0,1026238.0,2022-05-21 04:20:58,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-18,179477,275732,265834,42572,99287,11,7855,9135308,422917,6813741,...,908078,238842,8651,523119,10701796,657456,535,11819,320680,250007
2022-05-19,179597,275732,265841,42572,99287,11,7910,9135308,422917,6863970,...,908078,238855,8659,523174,10704524,657456,535,11819,320945,250206
2022-05-20,179624,275732,265847,42572,99287,11,7910,9135308,422917,6910455,...,908078,238868,8666,523238,10706111,657456,535,11819,320945,250469


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-18,7691,3497,6875,153,1900,0,138,128776,8623,7977,...,7215,1637,14,5713,43072,5659,0,2149,3983,5486
2022-05-19,7695,3497,6875,153,1900,0,138,128776,8623,8028,...,7215,1637,14,5715,43073,5659,0,2149,3984,5487
2022-05-20,7695,3497,6875,153,1900,0,138,128776,8623,8076,...,7215,1637,14,5716,43075,5659,0,2149,3984,5489


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-05-19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-05-20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/11/22,5/12/22,5/13/22,5/14/22,5/15/22,5/16/22,5/17/22,5/18/22,5/19/22,5/20/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,15853,15860,15863,15863,15863,15870,15871,15873,15885,15891
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,3745,3745,3745,3745,3745,3746,3746,3746,3747,3748
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,24181,24187,24193,24193,24193,24204,24208,24218,24225,24236
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,7680,7681,7682,7682,7682,7689,7689,7693,7694,7698
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,37858,37876,37888,37888,37888,37922,37933,37944,37966,37981


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-05-18,1306662,258039,6065,2038129,839433,9403268,1414790,793615,269818,49,...,2042775,6883580,938864,129854,18551,1747607,1544119,507678,1649615,157409
2022-05-19,1307291,258039,6065,2038129,839836,9407983,1416633,795492,270491,49,...,2042775,6888680,943368,129854,18551,1751443,1544119,508538,1652368,157409
2022-05-20,1307883,258039,6072,2038129,840151,9449915,1418901,797616,271294,49,...,2042775,6893736,943368,129854,18854,1755290,1552956,509251,1654808,157409


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-05-18       15873   55960    5691  6466  15025    2327   5070   32498   
2022-05-19       15885   55996    5691  6468  15031    2327   5071   32509   
2022-05-20       15891   56031    5692  6469  15039    2328   5073   32512   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-05-18         8516     5135  ...    6821   1916     8107     1927   
2022-05-19         8517     5135  ...    6821   1916     8107     1927   
2022-05-20         8518     5137  ...    6821   1916     8107     1927   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2022-05-18          11119  10116  5647          0     2363   1588  
2022-05-19          11119  10116  5647          0     2363   1588  
2022-05-20          11119  10116  5647          0     2363   1588  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-05-18,19633,1286,30,30259,11426,90904,12516,10922,2936,0,...,26267,88272,4761,654,113,20328,12833,6905,14514,1820
2022-05-19,19641,1286,30,30259,11426,90918,12515,10922,2936,0,...,26299,88291,4765,654,113,20332,12833,6912,14518,1820
2022-05-20,19645,1286,30,30259,11432,91044,12519,10926,2939,0,...,26311,88303,4765,654,113,20338,12863,6915,14531,1820


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-05-18         216     681      98  105    243      54    129     627   
2022-05-19         216     681      98  105    243      54    129     628   
2022-05-20         216     681      98  105    243      54    129     628   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-05-18          162       86  ...     144     41       79       27   
2022-05-19          162       86  ...     144     41       79       27   
2022-05-20          162       86  ...     144     41       79       27   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-05-18            126    16    39          0       44     18  
2022-05-19            126    16    39          0       44     18  
2022-05-20            126    16    39          0       44     18  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-18,0.000831,0.00016,0.000023,0.009868,0.0,0.0,0.007697,0.0,0.0,0.007986,...,0.0,0.000075,0.019324,0.000115,0.000171,0.0,0.0,0.0,0.000000,0.001037
2022-05-19,0.000669,0.00000,0.000026,0.000000,0.0,0.0,0.007002,0.0,0.0,0.007372,...,0.0,0.000054,0.000925,0.000105,0.000255,0.0,0.0,0.0,0.000826,0.000796
2022-05-20,0.000150,0.00000,0.000023,0.000000,0.0,0.0,0.000000,0.0,0.0,0.006772,...,0.0,0.000054,0.000808,0.000122,0.000148,0.0,0.0,0.0,0.000000,0.001051


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [61]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-18,0.00000,0.0,0.0,0.0,0.0,NaN,0.007299,0.0,0.0,0.006435,...,0.0,0.0,0.0,0.000175,0.000023,0.0,NaN,0.0,0.000000,0.000365
2022-05-19,0.00052,0.0,0.0,0.0,0.0,NaN,0.000000,0.0,0.0,0.006393,...,0.0,0.0,0.0,0.000350,0.000023,0.0,NaN,0.0,0.000251,0.000182
2022-05-20,0.00000,0.0,0.0,0.0,0.0,NaN,0.000000,0.0,0.0,0.005979,...,0.0,0.0,0.0,0.000175,0.000046,0.0,NaN,0.0,0.000000,0.000365


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [65]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-05-18,0.000418,0.007614,0.002811,0.003547,0.000551,0.000499,0.001345,0.002432,0.002303,0.0,...,0.003172,0.000713,0.000000,0.003012,0.006838,0.001838,0.003830,0.000867,0.001570,0.0
2022-05-19,0.000481,0.000000,0.000000,0.000000,0.000480,0.000501,0.001303,0.002365,0.002494,0.0,...,0.000000,0.000741,0.004797,0.000000,0.000000,0.002195,0.000000,0.001694,0.001669,0.0
2022-05-20,0.000453,0.000000,0.001154,0.000000,0.000375,0.004457,0.001601,0.002670,0.002969,0.0,...,0.000000,0.000734,0.000000,0.000000,0.016333,0.002197,0.005723,0.001402,0.001477,0.0


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-05-18      0.000126  0.000340  0.000527  0.000155  0.000400  0.003017   
2022-05-19      0.000756  0.000643  0.000000  0.000309  0.000399  0.000000   
2022-05-20      0.000378  0.000625  0.000176  0.000155  0.000532  0.000430   

Province_State                                          ... Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...    Park Platte   
2022-05-18      0.000000  0.000369  0.000470  0.000195  ...     0.0    0.0   
2022-05-19      0.000197  0.000338  0.000117  0.000000  ...     0.0    0.0   
2022-05-20      0.000394  0.000092  0.000117  0.000389  ...     0.0    0.0   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2022-05-18          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2022-05-19          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2022-05-20          0.0      0.0        0.0   0.0   0.0        NaN      0.0   

Province_State         
Admin2         Weston  
2022-05-18        0.0  
2022-05-19        0.0  
2022-05-20        0.0  

[3 rows x 3336 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-05-18,0.000000,0.014196,0.0,0.000959,0.000701,0.000187,0.00000,0.000000,0.000000,NaN,...,0.000076,0.000102,0.00000,0.0,0.0,-0.000098,0.001639,0.000290,0.000414,0.0
2022-05-19,0.000407,0.000000,0.0,0.000000,0.000000,0.000154,-0.00008,0.000000,0.000000,NaN,...,0.001218,0.000215,0.00084,0.0,0.0,0.000197,0.000000,0.001014,0.000276,0.0
2022-05-20,0.000204,0.000000,0.0,0.000000,0.000525,0.001386,0.00032,0.000366,0.001022,NaN,...,0.000456,0.000136,0.00000,0.0,0.0,0.000295,0.002338,0.000434,0.000895,0.0


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler   Calhoun   
2022-05-18         0.0     0.0     0.0  0.0    0.0     0.0    0.0  0.000000   
2022-05-19         0.0     0.0     0.0  0.0    0.0     0.0    0.0  0.001595   
2022-05-20         0.0     0.0     0.0  0.0    0.0     0.0    0.0  0.000000   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-05-18          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2022-05-19          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2022-05-20          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-05-18            0.0   0.0   0.0        NaN      0.0    0.0  
2022-05-19            0.0   0.0   0.0        NaN      0.0    0.0  
2022-05-20            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [73]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-18,0.000486,0.000162,0.000019,0.005017,0.0,0.0,0.005076,0.000234,0.000005,0.008250,...,0.001540,0.000059,0.011673,0.000082,0.000171,0.000074,0.0,0.0,0.000046,0.000982
2022-05-19,0.000577,0.000081,0.000023,0.002508,0.0,0.0,0.006039,0.000117,0.000003,0.007811,...,0.000770,0.000057,0.006299,0.000093,0.000213,0.000037,0.0,0.0,0.000436,0.000889
2022-05-20,0.000364,0.000041,0.000023,0.001254,0.0,0.0,0.003020,0.000059,0.000001,0.007292,...,0.000385,0.000056,0.003554,0.000108,0.000181,0.000019,0.0,0.0,0.000218,0.000970


In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [75]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-18,0.000045,1.000000e-07,0.0,-0.0,0.0,NaN,0.003650,0.000023,1.000000e-07,0.006381,...,0.000070,0.0,2.400000e-06,0.000102,0.000043,0.000088,NaN,0.0,9.000000e-07,0.000232
2022-05-19,0.000283,1.000000e-07,0.0,-0.0,0.0,NaN,0.001825,0.000012,1.000000e-07,0.006387,...,0.000035,0.0,1.200000e-06,0.000226,0.000033,0.000044,NaN,0.0,1.260000e-04,0.000207
2022-05-20,0.000141,0.000000e+00,0.0,-0.0,0.0,NaN,0.000913,0.000006,0.000000e+00,0.006183,...,0.000017,0.0,6.000000e-07,0.000200,0.000040,0.000022,NaN,0.0,6.300000e-05,0.000286


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [77]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-18,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-05-19,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-05-20,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [79]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-05-18,0.000397,0.003832,0.003421,0.001784,0.000400,0.001484,0.001363,0.003543,0.002242,0.0,...,0.001594,0.000579,0.000029,0.002783,0.007660,0.002045,0.002636,0.000987,0.001593,0.000605
2022-05-19,0.000439,0.001916,0.001711,0.000892,0.000440,0.000992,0.001333,0.002954,0.002368,0.0,...,0.000797,0.000660,0.002413,0.001391,0.003830,0.002120,0.001318,0.001340,0.001631,0.000303
2022-05-20,0.000446,0.000958,0.001432,0.000446,0.000408,0.002725,0.001467,0.002812,0.002668,0.0,...,0.000399,0.000697,0.001206,0.000696,0.010082,0.002158,0.003521,0.001371,0.001554,0.000151


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [81]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-05-18      0.000142  0.000417  0.000429  0.000281  0.000356  0.001623   
2022-05-19      0.000449  0.000530  0.000214  0.000295  0.000377  0.000812   
2022-05-20      0.000413  0.000578  0.000195  0.000225  0.000455  0.000621   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-05-18      0.000049  0.000385  0.000316  0.000243  ...  0.000739   
2022-05-19      0.000123  0.000362  0.000217  0.000121  ...  0.000370   
2022-05-20      0.000259  0.000227  0.000167  0.000255  ...  0.000185   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-05-18      0.000654  0.000316  0.000133   0.000701  0.001048  0.000180   
2022-05-19      0.000327  0.000158  0.000066   0.000351  0.000524  0.000090   
2022-05-20      0.000163  0.000079  0.000033   0.000175  0.000262  0.000045   

Province_State                                     
Admin2         Unassigned  Washakie        Weston  
2022-05-18           -1.0  0.000426 -1.200000e-06  
2022-05-19           -1.0  0.000213 -6.000000e-07  
2022-05-20           -1.0  0.000106 -3.000000e-07  

[3 rows x 3336 columns]

In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [83]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-05-18,0.000056,0.007145,0.000579,0.000485,0.000382,0.000388,0.000021,0.000229,0.000443,NaN,...,0.000052,0.000091,0.000002,0.001554,5.000000e-07,0.000091,0.001061,0.000482,0.000295,0.000416
2022-05-19,0.000232,0.003572,0.000290,0.000243,0.000191,0.000271,-0.000030,0.000115,0.000221,NaN,...,0.000635,0.000153,0.000421,0.000777,3.000000e-07,0.000144,0.000531,0.000748,0.000285,0.000208
2022-05-20,0.000218,0.001786,0.000145,0.000121,0.000358,0.000829,0.000145,0.000240,0.000622,NaN,...,0.000546,0.000145,0.000210,0.000389,1.000000e-07,0.000219,0.001434,0.000591,0.000590,0.000104


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [85]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State Alabama                                                    \
Admin2         Autauga   Baldwin Barbour      Bibb Blount Bullock Butler   
2022-05-18         0.0  0.000023     0.0  0.000075    0.0     0.0    0.0   
2022-05-19         0.0  0.000012     0.0  0.000038    0.0     0.0    0.0   
2022-05-20         0.0  0.000006     0.0  0.000019    0.0     0.0    0.0   

Province_State                              ...   Wyoming                   \
Admin2           Calhoun Chambers Cherokee  ...      Park Platte  Sheridan   
2022-05-18      0.000013      0.0      0.0  ...  0.000014    0.0  0.003205   
2022-05-19      0.000804      0.0      0.0  ...  0.000007    0.0  0.001603   
2022-05-20      0.000402      0.0      0.0  ...  0.000003    0.0  0.000801   

Province_State                                                                
Admin2         Sublette    Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-05-18          0.0  1.000000e-07   0.0   0.0       -1.0      0.0    0.0  
2022-05-19          0.0  1.000000e-07   0.0   0.0       -1.0      0.0    0.0  
2022-05-20          0.0  0.000000e+00   0.0   0.0       -1.0      0.0    0.0  

[3 rows x 3336 columns]

In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [87]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [89]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [90]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [91]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [92]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


#### 